In [12]:
import numpy as np
from keras.utils import np_utils
from keras.models import Model, Sequential, load_model
from keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, Reshape, Dropout
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.vgg16 import VGG16
# from keras.applications.resnet50 import ResNet50
from sklearn.metrics import confusion_matrix, classification_report
import itertools
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os

In [13]:
df = pd.read_csv('movie_frames.csv')
df.rename(columns={'Unnamed: 0': 'index',
                   '0': 'image_path'},
          inplace=True)

In [15]:
ids = []
for i in df['image_path']:
    ids.append(i.split('\\')[-1].split('-')[0])

df['id'] = ids
df['id'] = df['id'].astype(int)

In [17]:
model = VGG16(weights="imagenet", include_top=True, input_shape=(224, 224, 3))
out = model.layers[-2].output
model_final = Model(inputs=model.input, outputs=out)

In [18]:
model_final.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0   

In [19]:
def load_image(path):
    img = cv2.imread(path)
    img = cv2.resize(img, (224, 224))
    return img


In [20]:
errors = []
for i in range(0,6478):
    try:
        n_path = os.getcwd() + '\\movie_features\\' + str(i) + '.npy'
        image_list = list(df['image_path'][df['id'] == i])
        print(i, 'is running')
        if len(image_list) == 80:
            images = np.zeros((len(image_list), 224, 224, 3))
            for j in range(len(image_list)):
                img = load_image(image_list[j])
                images[j] = img
                
            images = np.array(images)
            feats = model_final.predict(images, batch_size=128)
            img_feats = np.array(feats)
            np.save(n_path,img_feats)
            print(i, 'is done')
    except:
        print('Error in: ', i)
        errors.append(i)

0 is running
1 is running
1 is done
2 is running
2 is done
3 is running
3 is done
4 is running
4 is done
5 is running
5 is done
6 is running
6 is done
7 is running
7 is done
8 is running
8 is done
9 is running
9 is done
10 is running
10 is done


In [ ]:
e_df = pd.DataFrame(errors)
e_df.to_csv('errors.csv')